# hydro_timeseries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in csv outputs from PG

existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
#new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
#potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
#all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
#fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
#fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
# load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [3]:
def hydro_timeseries(existing_gen, existing_variability, period_list):
    # List of hydro technologies in technology column from existing gen
    hydro_list = ['Conventional Hydroelectric', 'Hydroelectric Pumped Storage', 'Small Hydroelectric']

    # filter existing gen to just hydro technologies
    hydro_df = existing_gen.copy()
    hydro_df['index'] = hydro_df.index
    hydro_df = hydro_df[hydro_df['technology'].isin(hydro_list)]
    hydro_indx = hydro_df['index'].to_list()

    # get existing variability for the hydro technologies
    hydro_variability = existing_variability.copy()
    hydro_variability = hydro_variability.iloc[:, hydro_indx] # hydro hourly for 1 year

    # get cap size for each hydro tech
    hydro_cap_size = hydro_df['Cap_size'].to_list() # cap size for each hydro

    # multiply cap size by hourly
    for i in range(len(hydro_cap_size)):
        hydro_variability.iloc[:, i] = hydro_variability.iloc[:, i].apply(lambda x: x* hydro_cap_size[i])

    hydro_transpose = hydro_variability.transpose()

    month_hrs = [744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744]
    year_cumul = [744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760] # cumulative hours by month

    # split dataframe into months
    M1_df = hydro_transpose.iloc[:, 0:year_cumul[0]]
    M2_df = hydro_transpose.iloc[:, year_cumul[0]: year_cumul[1]]
    M3_df = hydro_transpose.iloc[:, year_cumul[1]: year_cumul[2]]
    M4_df = hydro_transpose.iloc[:, year_cumul[2]: year_cumul[3]]
    M5_df = hydro_transpose.iloc[:, year_cumul[3]: year_cumul[4]]
    M6_df = hydro_transpose.iloc[:, year_cumul[4]: year_cumul[5]]
    M7_df = hydro_transpose.iloc[:, year_cumul[5]: year_cumul[6]]
    M8_df = hydro_transpose.iloc[:, year_cumul[6]: year_cumul[7]]
    M9_df = hydro_transpose.iloc[:, year_cumul[7]: year_cumul[8]]
    M10_df = hydro_transpose.iloc[:, year_cumul[8]: year_cumul[9]]
    M11_df = hydro_transpose.iloc[:, year_cumul[9]: year_cumul[10]]
    M12_df = hydro_transpose.iloc[:, year_cumul[10]: year_cumul[11]]

    # get min and average for each month
    month_df = [M1_df, M2_df, M3_df, M4_df, M5_df, M6_df, M7_df, M8_df, M9_df, M10_df, M11_df, M12_df]
    month_names = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12']
    df_list = list()
    for i in range(len(month_df)):
        df = pd.DataFrame(hydro_transpose.index, columns=['hydro_project'])
        df['timeseries'] = month_names[i]
        df['hydro_min_flow_mw'] = month_df[i].min(axis=1).to_list()
        df['hydro_avg_flow_mw'] = month_df[i].mean(axis=1).to_list()
        df_list.append(df)
    hydro_final = pd.concat(df_list, axis=0)
    
    # get the index of existing gen for the hydro_project columnn (tie to GENERATION_PROJECTS)
    hydro_df['region_resource_cluster'] = hydro_df["region"]+ "_"+ hydro_df["Resource"]+ "_"+ hydro_df["cluster"
                                                                                                ].astype(str)
    hydro_index_dict = dict(zip(hydro_df['region_resource_cluster'].to_list(), hydro_df['index'].to_list()))
    hydro_final['hydro_project'] = hydro_final['hydro_project'].apply(lambda x: hydro_index_dict[x])
    # generation_project starts wtih 1 not 0
    hydro_final['hydro_project'] = hydro_final['hydro_project'].apply(lambda x: x+1)
    
    timeseries_list = list()
    for decade in period_list:
        df2 = hydro_final.copy()
        df2['timeseries'] = decade +'_'+ df2['timeseries']
        timeseries_list.append(df2)
    hydro_final_df = pd.concat(timeseries_list, axis=0)
    
    
    return hydro_final_df

In [4]:
period_list = ['2020', '2030', '2040', '2050']
hydro_timeseries_table = hydro_timeseries(existing_gen, existing_variability, period_list)
hydro_timeseries_table

,hydro_project,timeseries,hydro_min_flow_mw,hydro_avg_flow_mw
0,5,2020_M1,0.36414,0.408807
1,6,2020_M1,0.78030,0.876014
2,7,2020_M1,2.60100,2.920047
3,8,2020_M1,2.60100,2.920047
4,9,2020_M1,2.60100,2.920047
...,...,...,...,...
1263,4273,2050_M12,0.30030,0.347591
1264,4274,2050_M12,0.30030,0.347591
1265,4275,2050_M12,1.26126,1.459883
1266,4276,2050_M12,21.00000,21.000000


In [5]:
hydro_timeseries_table.to_csv(r'SWITCH_Inputs\hydro_timeseries.csv', index = False)